In [1]:
import pandas as pd
import gspread
import requests

token = 'AF5y90PW7vmJALond4SEGVBFwmt3Jhj7Ba7oclAeXL4'

In [2]:
sa = gspread.service_account(filename='geprek-wani-c700b4b8002f.json')

In [3]:
sheet = sa.open('Wani Spreadsheet')
worksheet = sheet.worksheet('Pesanan')

In [4]:
database = sheet.worksheet('Database Harga')
df_harga = pd.DataFrame(database.get_all_records())
df_harga.head()

,Menu,Harga
0,Nasi Ayam Geprek,12000
1,Nasi Ayam Bakar,15000
2,Nasi Ayam Richeese,15000
3,Nasi Ayam Mentega,15000
4,Nasi Ayam Goreng Lengkuas,15000


In [5]:
df = pd.DataFrame(worksheet.get_all_records())
df.rename(columns={'Request cabe GEPREK': 'Cabe', 'Sambal (+ 2.000)': 'Sambal'}, inplace=True)
df.head()

,Timestamp,Nama,Makanan Utama,Cabe,Camilan,Minuman,Sambal
0,11/05/2023 08:35:52,Abel,Nasi Ayam Mentega,,,,
1,11/05/2023 08:39:47,MW,Nasi Goreng Kulit,,,,
2,11/05/2023 08:41:07,Rey,Indomie Telur Geprek,0,,Es teh jumbo,
3,11/05/2023 08:41:27,Rey,Indomie Telur Geprek,0,,,
4,11/05/2023 08:41:40,Tot,Nasi Goreng Jawa,,,Es teh jumbo,


In [6]:
invoice = sheet.worksheet('Invoice')
df_invoice = pd.DataFrame(invoice.get_all_records())
df_invoice.head()

""


In [7]:
# export all Nama from df to df_invoice
df_invoice['Nama'] = df['Nama']

# merge df and df_invoice
df_invoice = pd.merge(df_invoice, df, on='Nama')

df_invoice['Status'] = 'UNPAID'
#drop timestamp
df_invoice.drop(['Timestamp'], axis=1, inplace=True)
#drop duplicate
df_invoice.drop_duplicates(subset=['Nama'], inplace=True)

df_invoice['Amount'] = 0

# get amount from iterating each row of df_invoice and get the price from df_harga
for index, row in df_invoice.iterrows():
    amount = 0
    for column in df_invoice.columns:
        if column == 'Nama' or column == 'Status':
            continue
        if row[column] != 0:
            harga_row = df_harga[df_harga['Menu'] == column]
            if not harga_row.empty:
                harga = harga_row.iloc[0]['Harga']
                amount += harga * row[column]
    df_invoice.at[index, 'Amount'] = amount

df_invoice.head()

,Nama,Makanan Utama,Cabe,Camilan,Minuman,Sambal,Status,Amount
0,Abel,Nasi Ayam Mentega,,,,,UNPAID,0
1,MW,Nasi Goreng Kulit,,,,,UNPAID,0
2,Rey,Indomie Telur Geprek,0,,Es teh jumbo,,UNPAID,0
6,Tot,Nasi Goreng Jawa,,,Es teh jumbo,,UNPAID,0
10,Timothy,Nasi Ayam Bakar,,,,,UNPAID,0


In [8]:
# update sheet
invoice.update([df_invoice.columns.values.tolist()] + df_invoice.values.tolist())

{'spreadsheetId': '1c9FkUpQxRoe4K0x_SyzNIxHs7AGsNREmIOOM6v0U41o',
 'updatedRange': 'Invoice!A1:H17',
 'updatedRows': 17,
 'updatedColumns': 8,
 'updatedCells': 136}